<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill for Stock Market Data</span>




## Imports

In [8]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import util
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

### TO WIPE OUT ALL OF FEATURES AND MODELS, run the cell below

In [9]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""
# proj = hopsworks.login()
# util.purge_project(proj)

---

## Alphavantage API Key

In [10]:
api_key_file = '../data/alphavantage-api-key.txt'
util.check_file_path(api_key_file)

with open(api_key_file, 'r') as file:
    ALPHAVANTAGE_API_KEY = file.read().rstrip()

File successfully found at the path: ../data/alphavantage-api-key.txt


## Hopsworks API Key


In [11]:
with open('../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login(project = 'StockPrediction', api_key_value=os.environ["HOPSWORKS_API_KEY"])

project

2025-01-06 11:52:58,202 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-06 11:52:58,211 INFO: Initializing external client
2025-01-06 11:52:58,214 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-06 11:53:00,744 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1205424


Project('StockPrediction', 'theresa.hoesl@t-online.de', 'Default project')

In [12]:
project.name

'StockPrediction'

In [13]:
secrets_api = hopsworks.get_secrets_api()
try:
    secrets_api.create_secret('AV_API_KEY', ALPHAVANTAGE_API_KEY)
except hopsworks.RestAPIError:
    ALPHAVANTAGE_API_KEY = secrets_api.get_secret("AV_API_KEY").value

### Validate that ALPHAVANTAGE_API_KEY works

In [15]:
try:
    sp_SPOT_df = util.get_stock_price('SPOT', ALPHAVANTAGE_API_KEY)
except hopsworks.RestAPIError:
    print("It looks like the AQI_API_KEY doesn't work for your sensor. Is the API key correct? Is the sensor URL correct?")

sp_SPOT_df.head()

,timestamp,price
0,2025-01-03,466.69


## Read CSV file into DataFrames

In [16]:
csv_file_SPOT="../data/daily_SPOT.csv"
csv_file_GOOGL="../data/daily_GOOGL.csv"
csv_file_BTC="../data/currency_daily_BTC_USD.csv"
csv_file_SandP="../data/daily_IVV.csv"

In [17]:
df_SPOT = pd.read_csv(csv_file_SPOT)
df_GOOGL = pd.read_csv(csv_file_GOOGL)
df_BTC = pd.read_csv(csv_file_BTC)
df_SandP = pd.read_csv(csv_file_SandP)

## Data cleaning

In [18]:
# extract closing price
df_SPOT.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)
df_GOOGL.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)
df_BTC.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)
df_SandP.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)

df_SPOT.rename(columns={'close': 'price'}, inplace=True)
df_GOOGL.rename(columns={'close': 'price'}, inplace=True)
df_BTC.rename(columns={'close': 'price'}, inplace=True)
df_SandP.rename(columns={'close': 'price'}, inplace=True)

df_SPOT['timestamp']=pd.to_datetime(df_SPOT['timestamp'])
df_GOOGL['timestamp']=pd.to_datetime(df_GOOGL['timestamp'])
df_BTC['timestamp']=pd.to_datetime(df_BTC['timestamp'])
df_SandP['timestamp']=pd.to_datetime(df_SandP['timestamp'])

In [19]:
type(df_SPOT['timestamp'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [20]:
df_SPOT.dropna(inplace=True)
df_GOOGL.dropna(inplace=True)
df_BTC.dropna(inplace=True)
df_SandP.dropna(inplace=True)

## Define Data Validation Rules 


In [21]:
import great_expectations as ge

# Create an Expectation Suite
sp_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="sp_expectation_suite"
)

# Add an expectation to check that all prices are >= 0
sp_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "price",
            "min_value": 0.01,  # Set minimum value to 0.01 to exclude 0 and negatives
        }
    )
)


{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "price", "min_value": 0.01}, "meta": {}}

## Connect to Hopsworks

In [22]:
fs = project.get_feature_store() 

## Create the Feature Groups and insert the DataFrames in them

In [23]:
SPOT_fg = fs.get_or_create_feature_group(
    name='spot',
    description='Spotify Stock Prices',
    version=1,
    primary_key=['timestamp'],
    event_time="timestamp",
    expectation_suite=sp_expectation_suite
)

GOOGL_fg = fs.get_or_create_feature_group(
    name='googl',
    description='Google Stock Prices',
    version=1,
    primary_key=['timestamp'],
    event_time="timestamp",
    expectation_suite=sp_expectation_suite
)

BTC_fg = fs.get_or_create_feature_group(
    name='btc',
    description='Bitcoin Prices',
    version=1,
    primary_key=['timestamp'],
    event_time="timestamp",
    expectation_suite=sp_expectation_suite
)

SandP_fg = fs.get_or_create_feature_group(
    name='sandp',
    description='S&P 500 Index Prices',
    version=1,
    primary_key=['timestamp'],
    event_time="timestamp",
    expectation_suite=sp_expectation_suite
)

In [24]:
SPOT_fg.insert(df_SPOT)
GOOGL_fg.insert(df_GOOGL)
BTC_fg.insert(df_BTC)
SandP_fg.insert(df_SandP)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1393604
2025-01-06 11:53:08,717 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1393604


Uploading Dataframe: 100.00% |██████████| Rows 1689/1689 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: spot_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1205424/jobs/named/spot_1_offline_fg_materialization/executions
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1394635
2025-01-06 11:53:26,829 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1394635


Uploading Dataframe: 100.00% |██████████| Rows 5117/5117 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: googl_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1205424/jobs/named/googl_1_offline_fg_materialization/executions
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1394636
2025-01-06 11:53:46,084 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1394636


Uploading Dataframe: 100.00% |██████████| Rows 350/350 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: btc_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1205424/jobs/named/btc_1_offline_fg_materialization/executions
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1393605
2025-01-06 11:54:07,964 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1393605


Uploading Dataframe: 100.00% |██████████| Rows 6195/6195 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: sandp_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1205424/jobs/named/sandp_1_offline_fg_materialization/executions


(Job('sandp_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "price",
           "min_value": 0.01
         },
         "meta": {
           "expectationId": 696079
         }
       },
       "result": {
         "observed_value": 68.19,
         "element_count": 6195,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-06T10:54:07.000964Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsuccessful_expectations": 0,
     "success_pe

#### Enter a description for each feature in the Feature Group

In [25]:
SPOT_fg.update_feature_description("timestamp", "Day of data")
SPOT_fg.update_feature_description("price", "closing price")

GOOGL_fg.update_feature_description("timestamp", "Day of data")
GOOGL_fg.update_feature_description("price", "closing price")

BTC_fg.update_feature_description("timestamp", "Day of data")
BTC_fg.update_feature_description("price", "closing price")

SandP_fg.update_feature_description("timestamp", "Day of data")
SandP_fg.update_feature_description("price", "closing price")

## Sentiment Backfill

### Getting the Data from the API

**NOTE:** `"sentiment_score_definition": "x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish"`

##### For Spotify

In [26]:
company_code = 'SPOT'

sentiment_api_url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&limit=1000&time_from=20180403T0000&tickers={company_code}&apikey={ALPHAVANTAGE_API_KEY}"
response = requests.get(sentiment_api_url)
spot_sentim_data = response.json()

In [27]:
spot_sentim_df = pd.read_json(json.dumps(spot_sentim_data))
spot_sentim_df["time_published"] = spot_sentim_df["feed"].apply(lambda x: x.get("time_published"))
spot_sentim_df["overall_sentiment_score"] = spot_sentim_df["feed"].apply(lambda x: x.get("overall_sentiment_score"))
spot_sentim_df.drop(columns=['feed','items','sentiment_score_definition','relevance_score_definition'], inplace=True)
spot_sentim_df["date_published"] = pd.to_datetime(spot_sentim_df["time_published"]).dt.date
spot_sentim_df.drop(columns=['time_published'], inplace=True)

In [28]:
# Convert the timestamp column in df_SPOT to date format
df_SPOT['date'] = df_SPOT['timestamp'].dt.date

# Merge the dataframes on the date columns
merged_df = pd.merge(spot_sentim_df, df_SPOT, left_on='date_published', right_on='date')

# Drop the extra date column
merged_df.drop(columns=['date','price','date_published'], inplace=True)

# Group by the timestamp and calculate the mean of sentiment scores
merged_df = merged_df.groupby('timestamp').mean().reset_index()

# Reindex merged_df to include all dates from df_SPOT
all_dates = pd.DataFrame(df_SPOT['timestamp'])
merged_df = all_dates.merge(merged_df, on='timestamp', how='left')

# Fill NaN values with 0
merged_df['overall_sentiment_score'].fillna(0, inplace=True)
spot_sentim_df = merged_df


##### For Google

In [29]:
company_code = 'GOOG'

sentiment_api_url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&limit=1000&time_from=20040819T0000&tickers={company_code}&apikey={ALPHAVANTAGE_API_KEY}"
response = requests.get(sentiment_api_url)
googl_sentim_data = response.json()

In [30]:
googl_sentim_df = pd.read_json(json.dumps(googl_sentim_data))
googl_sentim_df["time_published"] = googl_sentim_df["feed"].apply(lambda x: x.get("time_published"))
googl_sentim_df["overall_sentiment_score"] = googl_sentim_df["feed"].apply(lambda x: x.get("overall_sentiment_score"))
googl_sentim_df.drop(columns=['feed','items','sentiment_score_definition','relevance_score_definition'], inplace=True)
googl_sentim_df["date_published"] = pd.to_datetime(googl_sentim_df["time_published"]).dt.date
googl_sentim_df.drop(columns=['time_published'], inplace=True)

In [31]:
# Convert the timestamp column in df_SPOT to date format
df_GOOGL['date'] = df_GOOGL['timestamp'].dt.date

# Merge the dataframes on the date columns
merged_df = pd.merge(googl_sentim_df, df_GOOGL, left_on='date_published', right_on='date')

# Drop the extra date column
merged_df.drop(columns=['date','price','date_published'], inplace=True)

# Group by the timestamp and calculate the mean of sentiment scores
merged_df = merged_df.groupby('timestamp').mean().reset_index()

# Reindex merged_df to include all dates from df_SPOT
all_dates = pd.DataFrame(df_GOOGL['timestamp'])
merged_df = all_dates.merge(merged_df, on='timestamp', how='left')

# Fill NaN values with 0
merged_df['overall_sentiment_score'].fillna(0, inplace=True)
googl_sentim_df = merged_df


##### For BTC

In [32]:
company_code = 'CRYPTO:BTC'

sentiment_api_url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&limit=1000&time_from=20240103T0000&tickers={company_code}&apikey={ALPHAVANTAGE_API_KEY}"
response = requests.get(sentiment_api_url)
btc_sentim_data = response.json()

In [33]:
btc_sentim_df = pd.read_json(json.dumps(btc_sentim_data))
btc_sentim_df["time_published"] = btc_sentim_df["feed"].apply(lambda x: x.get("time_published"))
btc_sentim_df["overall_sentiment_score"] = btc_sentim_df["feed"].apply(lambda x: x.get("overall_sentiment_score"))
btc_sentim_df.drop(columns=['feed','items','sentiment_score_definition','relevance_score_definition'], inplace=True)
btc_sentim_df["date_published"] = pd.to_datetime(btc_sentim_df["time_published"]).dt.date
btc_sentim_df.drop(columns=['time_published'], inplace=True)

In [34]:
# Convert the timestamp column in df_SPOT to date format
df_BTC['date'] = df_BTC['timestamp'].dt.date

# Merge the dataframes on the date columns
merged_df = pd.merge(btc_sentim_df, df_BTC, left_on='date_published', right_on='date')

# Drop the extra date column
merged_df.drop(columns=['date','price','date_published'], inplace=True)

# Group by the timestamp and calculate the mean of sentiment scores
merged_df = merged_df.groupby('timestamp').mean().reset_index()

# Reindex merged_df to include all dates from df_SPOT
all_dates = pd.DataFrame(df_BTC['timestamp'])
merged_df = all_dates.merge(merged_df, on='timestamp', how='left')

# Fill NaN values with 0
merged_df['overall_sentiment_score'].fillna(0, inplace=True)
btc_sentim_df = merged_df

### Creating an Expectation Suite

In [35]:
sentim_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="sentim_expectation_suite"
)

# Add an expectation to check that all sentiment scores are between -3.5 to +3.5
sentim_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "overall_sentiment_score",
            "min_value": -3.5,
            "max_value": 3.5
        }
    )
)

{"expectation_type": "expect_column_values_to_be_between", "kwargs": {"column": "overall_sentiment_score", "min_value": -3.5, "max_value": 3.5}, "meta": {}}

### Creating Feature Groups and Inserting the Dataframes in them

In [36]:
spot_sentim_fg = fs.get_or_create_feature_group(
    name='spotify_sentiment',
    description='Spotify Sentiment Scores',
    version=1,
    primary_key=['timestamp'],
    event_time="timestamp",
    expectation_suite=sentim_expectation_suite
)

googl_sentim_fg = fs.get_or_create_feature_group(
    name='google_sentiment',
    description='Google Sentiment Scores',
    version=1,
    primary_key=['timestamp'],
    event_time="timestamp",
    expectation_suite=sentim_expectation_suite
)

btc_sentim_fg = fs.get_or_create_feature_group(
    name='bitcoin_sentiment',
    description='Bitcoin Sentiment Scores',
    version=1,
    primary_key=['timestamp'],
    event_time="timestamp",
    expectation_suite=sentim_expectation_suite
)

In [38]:
spot_sentim_fg.insert(spot_sentim_df)
googl_sentim_fg.insert(googl_sentim_df)
btc_sentim_fg.insert(btc_sentim_df)

2025-01-06 11:56:09,866 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1393606


Uploading Dataframe: 100.00% |██████████| Rows 1689/1689 | Elapsed Time: 00:02 | Remaining Time: 00:00


2025-01-06 11:56:22,462 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1393607


Uploading Dataframe: 100.00% |██████████| Rows 5117/5117 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: google_sentiment_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1205424/jobs/named/google_sentiment_1_offline_fg_materialization/executions
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1393608
2025-01-06 11:56:43,843 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1393608


Uploading Dataframe: 100.00% |██████████| Rows 350/350 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: bitcoin_sentiment_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1205424/jobs/named/bitcoin_sentiment_1_offline_fg_materialization/executions


(Job('bitcoin_sentiment_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_values_to_be_between",
         "kwargs": {
           "column": "overall_sentiment_score",
           "min_value": -3.5,
           "max_value": 3.5
         },
         "meta": {
           "expectationId": 696082
         }
       },
       "result": {
         "element_count": 350,
         "missing_count": 0,
         "missing_percent": 0.0,
         "unexpected_count": 0,
         "unexpected_percent": 0.0,
         "unexpected_percent_total": 0.0,
         "unexpected_percent_nonmissing": 0.0,
         "partial_unexpected_list": []
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-06T10:56:43.000843Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "ex

#### Adding Description of Column Values

In [39]:
spot_sentim_fg.update_feature_description("timestamp", "Day of News Articles")
spot_sentim_fg.update_feature_description("overall_sentiment_score", "Average of News Sentiment Scores")

googl_sentim_fg.update_feature_description("timestamp", "Day of News Articles")
googl_sentim_fg.update_feature_description("overall_sentiment_score", "Average of News Sentiment Scores")

btc_sentim_fg.update_feature_description("timestamp", "Day of News Articles")
btc_sentim_fg.update_feature_description("overall_sentiment_score", "Average of News Sentiment Scores")